Loading all necessary libraries

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import PIL
import pathlib

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Dropout, Flatten,Activation, BatchNormalization,MaxPooling2D, Input
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array





# Define image size and batch size
IMG_SIZE = 128
BATCH_SIZE = 32




Reading dataset 

In [3]:
import glob

image_dir = r'dataset\PetImages'   #add your path


pattern = os.path.join(image_dir, '*/*.jpg')


image_files = glob.glob(pattern)


print(image_files[:5])

['C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\0.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\1.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\10.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\100.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\1000.jpg']


reading dog images

In [4]:
dogs_pattern = os.path.join(image_dir, 'Dog/*')

dogs = glob.glob(dogs_pattern)


print(dogs[:5])

['C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Dog\\0.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Dog\\1.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Dog\\10.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Dog\\100.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Dog\\1000.jpg']


reading cat images

In [5]:
cats_pattern = os.path.join(image_dir, 'Cat/*')


cats = glob.glob(cats_pattern)

print(cats[:5])

['C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\0.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\1.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\10.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\100.jpg', 'C:\\Users\\aimen\\Desktop\\internshipProgram\\week5\\customerCNN\\dataset\\PetImages\\Cat\\1000.jpg']


In [6]:
pattern = os.path.join(image_dir, '*/*.jpg')

image_count = len(glob.glob(pattern))

print(image_count)

25000


In [ ]:
cats_pattern = os.path.join(image_dir, 'Cat/*')
dogs_pattern = os.path.join(image_dir, 'Dog/*')


images_dict = {
    'cats': glob.glob(cats_pattern),
    'dogs': glob.glob(dogs_pattern),
}

labels_dict = {
    'cats': 0,
    'dogs': 1,
}


print(images_dict['cats'][:5])
print(images_dict['dogs'][:5])

Setting our features and label varaible

In [ ]:
count = 0
X=[]
Y=[]
for pet_name, images in images_dict.items():
    #print(pet_name)
    for image in images:
        img = cv2.imread(str(image))
        if img is None:         
            continue
            
        elif ((img.shape[0] >= IMG_SIZE) and  (img.shape[1] >=IMG_SIZE)):
            resized_img = cv2.resize(img,(IMG_SIZE, IMG_SIZE))
            X.append(resized_img)
            Y.append(labels_dict[pet_name])
        else:
          
            continue

Converting to numpy array

In [9]:
X = np.array(X)
Y = np.array(Y)

Splitting data into training and testing

In [10]:
# train test split using dataframe
from sklearn.model_selection import train_test_split 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)
print(len(X_train),len(Y_train))
print(len(X_test),len(Y_test))

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

18327 18327
6110 6110
(18327, 128, 128, 3) (18327,)
(6110, 128, 128, 3) (6110,)


In [12]:
# Define the model
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    Conv2D(128, (3, 3), padding="same", activation="relu"),
    BatchNormalization(),      #applied on every activation function
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(8, (3, 3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 8)      │         2,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,085,821 (4.14 MB)

 Trainable params: 361,785 (1.38 MB)

 Non-trainable params: 464 (1.81 KB)

 Optimizer params: 723,572 (2.76 MB)

In [13]:


# Train the model
history=model.fit(X_train, Y_train, epochs=10, batch_size=32)


Epoch 1/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 3030s 5s/step - accuracy: 0.6573 - loss: 0.6656
Epoch 2/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 2806s 5s/step - accuracy: 0.7983 - loss: 0.4350
Epoch 3/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1724s 3s/step - accuracy: 0.8428 - loss: 0.3516
Epoch 4/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1866s 3s/step - accuracy: 0.8816 - loss: 0.2767
Epoch 5/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 2378s 4s/step - accuracy: 0.9167 - loss: 0.2065
Epoch 6/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 5420s 9s/step - accuracy: 0.9399 - loss: 0.1476
Epoch 7/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 2194s 4s/step - accuracy: 0.9659 - loss: 0.0958
Epoch 8/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1861s 3s/step - accuracy: 0.9763 - loss: 0.0659
Epoch 9/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1853s 3s/step - accuracy: 0.9864 - loss: 0.0453
Epoch 10/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1766s 3s/step - accuracy: 0.9852 - loss: 0.0433


In [16]:
accuracy = history.history['accuracy']
print('accuracy:', accuracy)

accuracy: [0.7123370170593262, 0.8026409149169922, 0.8453647494316101, 0.8813772201538086, 0.9124788641929626, 0.9356687068939209, 0.9613139033317566, 0.9711900353431702, 0.9817755222320557, 0.9821029305458069]


In [25]:
#saving weights
model.save_weights("/model_weights.weights.h5")  #add path to save the weights

In [27]:
#save the entire model
model.save("/full_model.keras")  #add path to save weights

-save weights
-validation accuracy ; has to be close to train accuray but not to less(overfitting) and not(underfitting) 